In [1]:
#put unitary representations of the symmetric group over GF(q^2) together to form unitary DFT
#if each rep'n is unitary, and we use normalizing factors \sqrt{d_\rho/|G|}, the overall matrix should be unitary

In [3]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(GF(q**2),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [4]:
def invariant_symmetric_bilinear_matrix(q,partition):
    """
    Computes the matrix of a S_n-invariant symmetric bilinear form.

    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 

    The equations are \rho(g)^T*U*\overline{\rho(g)} = \lambda_g*U where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}.

    The variables for U can be extracted to yield a matrix over GF(q^2) for each g.
    
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.

    Note: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    
    """

    # Define the group G and its rep'n as a Specht module, dimension
    n = sum(partition)
    SGA = SymmetricGroupAlgebra(GF(q^2), n)
    SM = SGA.specht_module(partition)
    G = SGA.group()
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(GF(q^2), 'u', d_rho^2)
    U_vars = R.gens()  # List of variable generators for U
    U = Matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):

        #compute \rho(g), transpose, conjugate
        rho_g = rho(Permutation(g))
        rho_g_T = rho_g.transpose()
        rho_g_conj = conjugate_pos_char(rho_g)
    
        # Compute lambda_g
        det_rho_g = det(rho_g)
        lambda_g = det_rho_g * (det_rho_g ** q)
    
        # Form the matrix equation \rho(g)^T*U*\overline{\rho(g)} = \lambda_g * U
        equation_matrix = rho_g_T*U*rho_g_conj - lambda_g * U
    
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
    
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
            
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
            
                # Append the row to the augmented system
                augmented_system.append(row)
    
        # Convert the augmented system to a matrix
        return Matrix(GF(q^2), augmented_system)

    #stack linear systems for each g in G
    total_system = matrix(GF(q^2),0,d_rho^2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    
    #return a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    U_mats = [matrix(GF(q^2),d_rho,d_rho,b) for b in null_space.basis()]

    #verify that a solution to the linear system satisfies the G-invariance property
    assert all(rho(g).transpose()*U_mats[0]*conjugate_pos_char(rho(g)) == U_mats[0] for g in G)
    
    return U_mats

In [5]:
#use libgap.eval for GAP evalutation of BaseChangeToCanonical using `forms` package
def unitary_change_of_basis(U,q):
    #for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
    if U.nrows() == 1 and U.ncols() == 1:
        if U[0,0] == 0:
            return matrix(GF(q**2), [[0]])  # Special case for 0
        z = GF(q**2).multiplicative_generator()
        k = U[0,0].log(z)  # Compute discrete log of u to the base z
        if k % (q+1) != 0:
            raise ValueError("Unable to factor: u.log(z) is not divisible by q+1, i.e. u is not in base field GF(q)")
        return matrix(GF(q**2), [[z**((k//(q+1))%(q-1))]])
    libgap.LoadPackage("forms")
    return matrix(GF(q^2),libgap.BaseChangeToCanonical(libgap([list(row) for row in U]).HermitianFormByMatrix(GF(q^2)))).inverse()

In [53]:
#define the Fourier coefficient at the rep'n specht_module corresponding to partition
def hat(f,partition,SGA,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        U = invariant_symmetric_bilinear_matrix(q,partition)[0]
        A = unitary_change_of_basis(U,q)
        A_star = conjugate_pos_char(A).transpose()
        sqrt_unitary_factor = sqrt(GF(q**2)(specht_module.dimension()/SGA.group().cardinality()))
        return sqrt_unitary_factor*sum(f(g)*A_star*rho(g)*A_star.inverse() for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [7]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
delta = lambda s: lambda t: 1 if t == s else 0 #delta function \delta_s(t)
def dft(SGA,unitary=False):
    fourier_transform = [flatten([hat(delta(g),partition,SGA,unitary).list() for partition in Partitions(SGA.group().degree())]) for g in SGA.group()]
    if unitary:
        return matrix(GF(q**2),fourier_transform).transpose()
    else:
        return matrix(fourier_transform).transpose()

In [11]:
n = 4; q = 11
SGA = SymmetricGroupAlgebra(GF(q^2), n)

In [54]:
unitary_dft = dft(SGA,unitary=True); unitary_dft

24 x 24 dense matrix over Finite Field in z2 of size 11^2 (use the '.str()' method to see the entries)

In [55]:
print(unitary_dft*conjugate_pos_char(unitary_dft).transpose())

[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  

In [22]:
[SGA.specht_module(la).dimension()**2 for la in Partitions(4)]

[1, 9, 4, 9, 1]

In [59]:
#note there are two distinct square roots for a nonzero element of GF(q**2)
#may need to carefully choose which square root to use
#perhaps even a fourth root of unity is necessary?
#see Travis' comment: we can probably just divide by \sqrt{c_i}
#if (v_i,v_j) = c_i\delta_{ij}, then v’_i = v_i / \pm \sqrt{c_i}
sqrt(GF(q**2)(SGA.specht_module([3,1]).dimension()/SGA.group().cardinality()))

3*z2 + 5